<a href="https://colab.research.google.com/github/JaswanthMannem/CREWAI/blob/main/crewai_4_tasks_event_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-openai

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Crew, Task
from langchain_openai import ChatOpenAI

In [4]:
from google.colab import userdata
key=userdata.get('OPENAI_API_KEY')
serp_key=userdata.get('SERP_API_KEY')

In [6]:
import os

openai_api_key = key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serp_key

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key),
)

In [9]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key),
)

In [10]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=ChatOpenAI(openai_api_key=openai_api_key),
)

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "tech conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/conferences/tech-conferences/
Snippet: Tech conferences events in San Francisco

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Western Addition, San Francisco',
 'booking_status': 'Available',
 'capacity': 200,
 'name': 'Imperial Ballroom at Hotel Kabuki'}


In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement Report

### Marketing Activities:
1. **Spokenote Elevates Event Engagement at RALLY Innovation Conference**
   - Showcase of technology transforming event engagement.
   - Personalized video connections for attendees.
   - [Read more here](https://spokenote.com/spokenote-elevates-event-engagement-rally-innovation-conference/?srsltid=AfmBOorientl632-q948OJbnZSPxqSxF4A4GCiExiTgIT3orYS4eeyKO)

2. **Enhance Attendee Engagement with complete Event Tech Guide**
   - Create value for attendees.
   - Build brand loyalty.
   - Maximize event success and ROI.
   - [Read more here](https://www.fielddrive.com/blog/maximizing-attendee-engagement-through-technology)

3. **Event Technology: A Comprehensive Guide To Success In 2024**
   - Innovative tools for streamlining operations.
   - Enhance attendee engagement.
   - Maximize event success.
   - [Read more here](https://taggbox.com/blog/event-technology/)

4. **Elevating Innovation: 5 Strategies for Successful Virtual Conferences**
   - Powerful platform for connecting and sharing knowledge.
   - Driving innovation across industries and geographies.
   - [Read more here](https://digitalpromise.org/2023/12/07/elevating-innovation-5-strategies-for-successful-virtual-conference/)

5. **15 Conference Marketing Ideas for 2024 that Really Work**
   - Identify target audience.
   - Segment audiences.
   - Optimize websites.
   - Encourage speakers to promote involvement.
   - Form influencer partnerships.
   - [Read more here](https://corp.kaltura.com/blog/conference-marketing/)

### Attendee Engagement:
1. **Attendee Engagement Technology: 11 Ideas for In-Person Events**
   - 11 ways to create more attendee engagement at in-person events.
   - Use of virtual and hybrid event technology.
   - Event experts from Encore.
   - [Read more here](https://www.encore-anzpac.com/11-ways-to-enhance-in-person-events-with-technology)

2. **Tech-Enhanced Event Storytelling - The Future of Attendee Engagement**
   - Leveraging Eventcombo's advanced tech stacks for effective storytelling.
   - Crafting effective storytelling strategies.
   - [Read more here](https://www.linkedin.com/pulse/tech-enhanced-event-storytelling-future-attendee-engagement-hdrjf)

3. **The Role of Event Technology in Enhancing Attendee Engagement**
   - Transformative impact of technology in event planning.
   - Enhancing attendee engagement and experience.
   - [Read more here](https://hub.theeventplannerexpo.com/technology/the-role-of-event-technology-in-enhancing-attendee-engagement-and-experience-2)

4. **13 Innovative Event Marketing Ideas**
   - Incorporating engaging onsite experiences like social media studios, virtual reality demos, interactive games, and live entertainment.
   - [Read more here](https://blog.walls.io/events/event-marketing-ideas/)

5. **20 Event Engagement Ideas to Wow Your Attendees**
   - Session Q&A's for attendee insights.
   - Social interactions with live chat for virtual sessions.
   - [Read more here](https://www.cvent.com/en/blog/events/20-event-engagement-ideas)

This comprehensive report showcases various marketing activities and attendee engagement strategies that can be implemented for the Tech Innovation Conference to ensure maximum exposure and participation.